## Dataset Diabetes

Este conjunto de datos proviene originalmente del Instituto Nacional de Diabetes y Enfermedades Digestivas y Renales. El objetivo del conjunto de datos es predecir si un paciente tiene o no diabetes, basándose en ciertas medidas diagnósticas incluidas en el conjunto de datos. Se impusieron varias restricciones en la selección de estas instancias de una base de datos más grande. En particular, todos los pacientes aquí son mujeres mayores de 21 años de herencia india Pima.

### Cargar los datos

Abrir el archivo `diabetes.csv` en la carpeta `1_datos`. Analizar su estructura y responder las siguientes preguntas:

*   ¿Cuántas instancias o ejemplos tiene el dataset?
*   ¿Cómo se llama la variable de salida o target? ¿Qué tipo de datos es?
*   ¿Cuántos atributos posee cada instancia o ejemplo?¿Puede detectar datos faltantes o mal registrados en algún atributo?
*   ¿Qué tipo de dato tiene cada atributo?

1) El dataset tiene 768 instancias.
2) La variable de salida se llama "Outcome". Es una variable categorica binaria, donde 1 indica que la persona tiene diabetes y 0 indica que no la tiene. Este tipo de datos es del tipo int.
3) Cada instancia tiene 9 atributos, más la columna "Outcome", lo que hace un total de 10 columnas (sin contar la columna Unnamed: 0 porque no se que significa, parece un tipo indice)

[falta responder la otra parte de la 3😝]

4) regnancies: int
Glucose: int
BloodPressure: int
SkinThickness: int
Insulin: int
BMI: float
DiabetesPedigreeFunction: float
Age: int
Outcome: int

In [17]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [18]:
datos = pd.read_csv('./1_datos/diabetes.csv')
datos


,Unnamed: 0,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,0,6,148,72.0,35.0,0.0,33.6,0.627,50,1
1,1,1,85,66.0,29.0,0.0,26.6,0.351,31,0
2,2,8,183,64.0,0.0,0.0,23.3,0.672,32,1
3,3,1,89,66.0,23.0,94.0,28.1,0.167,21,0
4,4,0,137,40.0,35.0,168.0,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...,...
763,763,10,101,76.0,48.0,180.0,32.9,0.171,63,0
764,764,2,122,70.0,27.0,0.0,36.8,0.340,27,0
765,765,5,121,72.0,23.0,112.0,26.2,0.245,30,0
766,766,1,126,60.0,0.0,0.0,30.1,0.349,47,1


In [19]:
datos= datos.drop(columns="Unnamed: 0") #Borro Unnamed: 0 ya que solo parece ser un ID

X = datos.drop(columns="Outcome")
y= datos["Outcome"]

In [20]:
datos.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,767.000000,756.000000,763.000000,767.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.195567,20.862434,80.322412,32.034289,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.206609,15.865314,115.439459,7.804050,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,63.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,36.000000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,128.500000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [21]:
faltante = datos.isna().sum() #si hay NA los sume, basicamente un ftable de R
print(faltante,"\n")

datos_faltantes = datos[datos.isnull().any(axis=1)]
datos_faltantes.head() #Algunos datos faltantes

Pregnancies                  0
Glucose                      0
BloodPressure                1
SkinThickness               12
Insulin                      5
BMI                          1
DiabetesPedigreeFunction     0
Age                          0
Outcome                      0
dtype: int64 



,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
7,10,115,NaN,NaN,NaN,35.3,0.134,29,0
9,8,125,96.0,NaN,NaN,NaN,0.232,54,1
10,4,110,92.0,NaN,NaN,37.6,0.191,30,0
11,10,168,74.0,NaN,NaN,38.0,0.537,34,1
12,10,139,80.0,NaN,NaN,27.1,1.441,57,0


### ⚠️
Antes de realizar cualquier procesamiento, no olvide de separar los datos en entrenamiento y prueba. Divida el conjunto de datos en entrenamiento y prueba utilizando train_test_split (con una proporción de 80% entrenamiento y 20% prueba) use 42 como semilla.

In [22]:
x_train, y_train, x_test, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

### Detección de valores atípicos y tratamiento de datos faltantes

- Analice con cuidado el rango de valores de los atributos. **¿Nota valores atípicos a simple vista en algún/algunos de los atributos?**

    -  Ejemplo: Un valor de insulina igual a 0 puede considerarse mal registrado o faltante.

- Los valores atípicos que detecte pueden considerarse datos faltantes. Utilice el método [replace](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.replace.html) de pandas para reemplazarlos por np.NaN.

In [23]:
datos["BMI"].sort_values().head(11)

#Claro ponele que insulin tiene valores 0, pero esto esta bien si hay personas sanas, no son todos enfermos de diabetes
#Yo supongo que con insulin se refiere a la insulina que se tienen que administrar los diabeticos. En las personas sanas obviamente va a ser 0 insulin inyectada

# "SkinThickness" esta no puede ser 0 asik si son datos faltantes tipo, no podes tener 0 de grosor de piel (hay valores que son 0)
# BMI (Índice de Masa Corporal) no podes tener 0


49      0.0
706     0.0
494     0.0
684     0.0
371     0.0
522     0.0
60      0.0
426     0.0
81      0.0
145     0.0
526    18.2
Name: BMI, dtype: float64

In [28]:
#Reemplazamos los valores por np.NaN 
datos.replace({"SkinThickness":0}, np.NaN, inplace=True)
datos.replace({"BMI":0}, np.NaN, inplace=True)

# Revisar los primeros datos para confirmar los cambios
datos.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72.0,35.0,0.0,33.6,0.627,50,1
1,1,85,66.0,29.0,0.0,26.6,0.351,31,0
2,8,183,64.0,NaN,0.0,23.3,0.672,32,1
3,1,89,66.0,23.0,94.0,28.1,0.167,21,0
4,0,137,40.0,35.0,168.0,43.1,2.288,33,1


Después de realizar el reemplazo, ¿Cuántos datos faltantes presenta cada atributo?

### Aclaración sobre la imputación de datos faltantes
Algo a tener en cuenta al momento de imputar datos faltantes es que los atributos pueden tener distribuciones diferentes según la salida o target (con diabetes/sin diabetes). Por ejemplo, una persona sana tendrá un valor de insulina en un rango distinto a una persona con diabetes.

- Implemente un transformer para imputar datos faltantes de forma condicional, dependiendo de la clase (diabetes/no diabetes). Para hacerlo, utilice la media o mediana correspondiente a cada clase.
- Utilice un Pipeline de scikit-learn para realizar la preparación completa de los datos, que incluya la imputación.


- Entrene al menos tres clasificadores distintos.
- Compare el desempeño de los modelos utilizando la métrica F1-score, ya que el dataset tiene un desbalance de clases (más casos sin diabetes que con diabetes). Puede evaluar otras métricas si lo desea. [sklearn metrics](https://scikit-learn.org/stable/modules/model_evaluation.html)